In [1]:
import pandas as pd
import folium

## 파일 불러오기

In [2]:
df1 = pd.read_csv('지진옥외대피소.csv',encoding = 'euckr')
df2 = pd.read_csv('지진해일대피소.csv',encoding = 'euckr')

In [3]:
df1.head()

,지역코드,시설일련번호,시도명,시군구명,시설명,상세주소,시설면적,경도,위도
0,4313000000,176,충청북도,충주시,충주교현초등학교 운동장,충청북도 충주시 교현동 339-1,8500,127.936678,36.974072
1,4313000000,145,충청북도,충주시,대가미공원,충청북도 충주시 교현동 714-0,32400,127.925197,36.980898
2,4313000000,146,충청북도,충주시,충주중앙중학교 운동장,충청북도 충주시 교현동 1088-0,4250,127.930701,36.981533
3,4311000000,6,충청북도,청주시,석성초등학교 운동장,충청북도 청주시 청원구 북이면 내둔리 344,7825,127.51374087453421,36.74848427755441
4,4313000000,138,충청북도,충주시,충주 관아공원,충청북도 충주시 성내동 232-2,3890,127.936202,36.970501


In [4]:
df2.head()

,일련번호,시도명,시군구명,대피지구명,대피장소명,주소,경도,위도,수용가능인원수,해변으로부터거리,대피소 분류명,내진적용여부,해발높이
0,608,경상북도,울진군,삼율지구3,7번국도 옆 공장부지,경상북도 울진군 후포면 금음리 28-4,129.436605,36.673303,500,200,공터,NaN,30
1,609,경상북도,울진군,금음지구2,풍경모텔뒷산,경상북도 울진군 후포면 금음리 산155-1,129.429702,36.664220,500,250,공터,NaN,24
2,610,경상북도,울진군,금음지구2,반소골길,경상북도 울진군 후포면 금음리 산259,129.398723,36.665528,300,100,공터,NaN,20
3,614,경상북도,울진군,삼율지구2,삼영아파트옥상,경상북도 울진군 후포면 울진대게로 24-7,129.442473,36.677247,400,150,민간건축물,NaN,20
4,615,경상북도,울진군,삼율지구3,공영하이츠옥상,경상북도 울진군 후포면 정실2길 24-1,129.439106,36.677606,500,300,민간건축물,NaN,35


## 지진옥외대피소 처리

- 수용가능인원수 추가
수용가능인원의 기준은 (군중밀집의 임계점은 m^2당 5명 so. 3명으로 잡고 반올림)   
//   참고자료 (# https://edition.cnn.com/interactive/2021/11/us/crowd-density-dangerous-warning-signs/ or https://www.donga.com/news/Society/article/all/20221031/116246753/2)


- 위도 경도 이상한 값 처리

In [5]:
df1['수용가능인원수'] = round(df1['시설면적']/3).astype(int)

In [6]:
df1.loc[df1['위도']=='35.15.02','위도'] = '35.1502'
df1.loc[df1['경도'] =='128.37.08','경도'] = '128.3708'
df1.loc[df1['위도']=='35.15.14','위도'] = '35.1514'
df1.loc[df1['경도']=='128.36.28','경도'] = '128.3628'
df1.loc[df1['경도']=='35.481726, 126.481198','경도'] = '126.481198'

## 지진옥외 / 지진해일 대피소 겹치는값 찾기

- 찾다보니 지진옥외 안에서 겹치는 값과 지진해일 에서 겹치는 값도 따로 있는 것을 발견함.

- 같은 데이터 안에서 겹치는 값을 지도를 통해 확인한 결과 -> 위치만 동일하고 (야산1,야산2) / (초등학교 , 중학교) 처럼 서로 다른 곳임을 발견했다.

-->> 이렇게 될 경우 상세주소의 unique값이 모두 달라야 하는데 겹치는 4곳의 장소를 발견하여 확인 해 본 결과 : 같은 곳임을 확인 하였고 이 4곳만 drop 처리.


- 우리가 찾는 것은 완전 겹치는 대피소를 찾고 있으므로 duplicated 할 때 같은 데이터 안에서 겹치는 값들은 제외하고 처리하기로 함.

In [7]:
df1[df1['상세주소'].duplicated(keep=False)]

,지역코드,시설일련번호,시도명,시군구명,시설명,상세주소,시설면적,경도,위도,수용가능인원수
83,4315000000,3,충청북도,제천시,금성초등학교 운동장,충청북도 제천시 금성면 구룡리 294-0,5953,128.16544612145643,37.06719875510558,1984
84,4315000000,49,충청북도,제천시,금성초등학교 운동장,충청북도 제천시 금성면 구룡리 294-0,5953,128.16548565149745,37.06723442591937,1984
324,4374500000,7,충청북도,증평군,형석중학교 운동장,충청북도 증평군 증평읍 미암리 603,7753,127.59314496391403,36.79279093756006,2584
325,4374500000,9,충청북도,증평군,형석고등학교 운동장,충청북도 증평군 증평읍 미암리 603,8400,127.59314496391403,36.79279093756006,2800
439,1159000000,19,서울특별시,동작구,영등포고등학교 운동장,서울특별시 동작구 대방동 13-2,5241,126.93383427187928,37.507988916849975,1747
...,...,...,...,...,...,...,...,...,...,...
10400,1129000000,35,서울특별시,성북구,길음중학교 운동장,서울특별시 성북구 길음동 635-7,2062,127.01838463022357,37.61363429459085,687
10404,1129000000,56,서울특별시,성북구,서울대학교 사범대학 부설 중학교 운동장,서울특별시 성북구 종암동 7-10,18452,127.03896757562556,37.5958036172146,6151
10405,1129000000,57,서울특별시,성북구,서울대학교 사범대학 부설 고등학교 운동장,서울특별시 성북구 종암동 7-10,12736,127.03781065509901,37.592726843326155,4245
10668,4719000000,137,경상북도,구미시,오상고등학교 운동장,경상북도 구미시 장천면 상장리 700,8500,128.49672620498637,36.12386556888312,2833


In [8]:
## 지진옥외대피소 겹치는 값 다른 곳인지 확인
# 4곳이 같은 것을 볼 수 있다.
df1[df1['상세주소'].duplicated(keep=False)]['시설명'].value_counts()

금성초등학교 운동장      2
빛가온중학교 운동장      2
자경2어린이공원        2
경기항공고등학교 운동장    2
미성어린이공원         1
               ..
서울예술고등학교 운동장    1
후창공원            1
광희중학교 운동장       1
무학중학교 운동장       1
오상중학교 운동장       1
Name: 시설명, Length: 215, dtype: int64

In [9]:
lis_duplicate = ['빛가온중학교 운동장','자경2어린이공원','경기항공고등학교 운동장']

In [10]:
map = folium.Map(location=[36, 127], zoom_start=7)
for s in lis_duplicate:
    lat1 = df1[df1['시설명'] == s]['경도'].values[0]
    lon1 =df1[df1['시설명'] == s]['위도'].values[0]
    lat2 = df1[df1['시설명'] == s]['경도'].values[1]
    lon2 =df1[df1['시설명'] == s]['위도'].values[1]
    marker = folium.Marker([lon1,lat1],popup='1').add_to(map)
    marker = folium.Marker([lon2,lat2],popup='2').add_to(map)

## 금성초등학교는 다른지역도 있어서 따로 처리.
marker = folium.Marker([37.06719875510558,128.16544612145643],popup='1').add_to(map)
marker = folium.Marker([37.06723442591937,128.16548565149745],popup='2').add_to(map)
map

#맵을 보면 같은 곳을 지칭 하는 것을 알 수 있다.

In [11]:
# 지진옥외대피소에서 겹치는 4곳 지우기.
df1 = df1.drop(index = df1[df1['시설명'] == '금성초등학교 운동장'][:1].index)
for g in lis_duplicate:
    df1 = df1.drop(index = df1[df1['시설명'] == g][:1].index)
df1.reset_index(inplace =  True)

In [12]:
## 지진해일대피소 겹치는 값 다른 곳인지 확인
df2[df2['주소'].duplicated(keep = False)]
df = df2[df2['주소'] == '강원도 양양군 현남면 남애리 1']

In [13]:
import folium
map = folium.Map(location=[36, 127], zoom_start=7)
for i in df.index:
    lat = df.loc[i, '위도']
    lon = df.loc[i, '경도']
    marker = folium.Marker([lat,lon]).add_to(map)
map

In [14]:
#지진해일 / 지진옥외 주소 합치기
df1_test = df1[['상세주소']]
df2_test = df2[['주소']]
df2_test = df2_test.rename({'주소':'상세주소'},axis =1)
df_address = pd.concat([df1_test,df2_test],axis = 0,ignore_index=True)

# df1_test 이랑 df2_test 겹치면 df2_test 결과 가져오려고 keep='first'
df_address_tot = df_address[df_address.duplicated(keep = 'first')]

In [15]:
df2[df2['주소'].duplicated()]

,일련번호,시도명,시군구명,대피지구명,대피장소명,주소,경도,위도,수용가능인원수,해변으로부터거리,대피소 분류명,내진적용여부,해발높이
15,623,경상북도,울진군,후포지구1,마을뒷산 고지대,경상북도 울진군 후포면 후포리 37,129.464335,36.688388,200,100,공터,NaN,19
284,269,강원도,양양군,남애지구,남애리 야산 2,강원도 양양군 현남면 남애리 1,128.785876,37.941682,200,200,공터,NaN,25
285,270,강원도,양양군,남애지구,남애리 야산 3,강원도 양양군 현남면 남애리 1,128.786258,37.945462,200,300,공터,NaN,25


In [16]:
# df1 / df2 각각의 겹치는 값들은 그대로 
df_address_tot = df_address_tot.drop(index = [10887,11156,11157])

In [17]:
## 지진옥외 / 지진해일 대피소 겹치는값.
## 10872는 df1의 row값
duplicated_list = df_address_tot[df_address_tot.index > 10872].index

In [18]:
duplicated_list

Int64Index([10918, 10919, 10924, 10951, 10955, 10965, 10967, 10968, 10989,
            10995, 11017, 11019, 11021, 11025, 11042, 11119, 11148, 11150,
            11208, 11230, 11234, 11263, 11271, 11280, 11283, 11291, 11293,
            11318, 11401, 11437],
           dtype='int64')

In [19]:
len(duplicated_list)

30

## 지진해일 / 지진옥외 대피소 합치기

- (겹치는 값을 df2기준으로 찾았기 때문에) df2가 밑에 있게 합쳐야 한다.
- 따라서 df2를 df1과 동일한 column명 / 동일한 column 개수가 되어야 한다.

- 또한 합친 후 인구 변화도 함께 볼 것이기 때문에 지진옥외 대피소와 지진해일 대피소의 '시군구명'이 겹치는 부분을 확인 한 결과 포항시 남구의 변화만 주면 이상없다.

In [20]:
df1.shape , df2.shape

((10872, 11), (648, 13))

In [21]:
df2.loc[df2['시군구명'] == '포항시남구','시군구명'] = '포항시 남구'

In [22]:
df2.rename({'대피장소명':'시설명','주소':'상세주소'}, axis=1,inplace = True)

In [23]:
df1 = df1[['시도명','시군구명','시설명','상세주소','경도','위도','수용가능인원수']]
df2 = df2[['시도명','시군구명','시설명','상세주소','경도','위도','수용가능인원수']]
df_total = pd.concat([df1,df2],axis = 0 , ignore_index= True)

## 지진해일 / 지진옥외 대피소 동일 주소 처리

- 값을 처리하다 보니 지진옥외대피소의 수용가능인원을 위에서 임의의 기준을 잡았는데 동일한 주소이면 수용가능인원이 동일해야 된다는 생각이 들었다.
- 확인 결과 값이 차이가 나서 지도화를 하여 비교해 본 결과
- 지진옥외대피소를 기준으로 보는 것이 더 정확한 것 같았다.
- 그래서 중복값을 지울 때 지진해일 데이터를 지웠다

In [24]:
## 지진옥외대피소랑 지진해일대피소 동일주소의 수용가능인원 확인

In [25]:
total_duplicate = df_total[df_total['상세주소'].duplicated(keep = False)]

In [26]:
total_duplicate

,시도명,시군구명,시설명,상세주소,경도,위도,수용가능인원수
323,충청북도,증평군,형석중학교 운동장,충청북도 증평군 증평읍 미암리 603,127.59314496391403,36.79279093756006,2584
324,충청북도,증평군,형석고등학교 운동장,충청북도 증평군 증평읍 미암리 603,127.59314496391403,36.79279093756006,2800
438,서울특별시,동작구,영등포고등학교 운동장,서울특별시 동작구 대방동 13-2,126.93383427187928,37.507988916849975,1747
440,서울특별시,동작구,영등포중학교 운동장,서울특별시 동작구 대방동 13-2,126.93286097282864,37.50812488139663,1120
501,전라북도,김제시,덕암정보고등학교,전라북도 김제시 서암동 5-0,126.88681629845776,35.8132608257477,1546
...,...,...,...,...,...,...,...
11291,경상북도,경주시,양남중학교 운동장,경상북도 경주시 양남면 하서리 651,129.460154,35.673865,1000
11293,경상북도,경주시,나산초등학교,경상북도 경주시 양남면 읍천리 17,129.4703,35.69788,1000
11318,경상북도,경주시,감포중·고등학교,경상북도 경주시 감포읍 오류리 478-2,129.506759,35.810771,1300
11401,경상북도,울진군,사동초등학교,경상북도 울진군 기성면 사동리 455,129.437804,36.830099,500


In [27]:
df1_duplicate = df1[df1['상세주소'].duplicated()].index
df2_duplicate = df2[df2['상세주소'].duplicated()].index

In [28]:
total_duplicate = total_duplicate.drop(index = list(df1_duplicate))
df2_duplicate = df2_duplicate+10872
duplicate_test = total_duplicate.drop(index = list(df2_duplicate))

In [29]:
# 1(600개 데이터) 이 지진해일대피소  2가 지진옥외대피소 ->
# 2를 임의로 설정 하였으니 틀린지 보면 된다.
duplicate_test_1 = duplicate_test[duplicate_test['상세주소'].duplicated(keep='first')]
duplicate_test_2 = duplicate_test[duplicate_test['상세주소'].duplicated(keep='last')]
duplicate_test_total = pd.merge(duplicate_test_1[['상세주소','수용가능인원수']],duplicate_test_2[['상세주소','수용가능인원수']], on='상세주소')

In [30]:
# 값을 보니 다른 부분이 많은 듯.
duplicate_test_total.head()

,상세주소,수용가능인원수_x,수용가능인원수_y
0,경상북도 울릉군 서면 남서리 30-3,400,632
1,경상북도 울릉군 울릉읍 도동리 223-8,400,1100
2,경상북도 울릉군 서면 남양리 586-2,400,989
3,울산광역시 동구 주전동 277,3939,595
4,울산광역시 동구 일산동 963,5212,2033


In [31]:
map = folium.Map(location=[36, 127], zoom_start=7)
for add in duplicate_test_total['상세주소']:
    lat1 = df1[df1['상세주소'] == add]['경도'].values[0]
    lon1 =df1[df1['상세주소'] == add]['위도'].values[0]
    lat2 = df2[df2['상세주소'] == add]['경도'].values[0]
    lon2 = df2[df2['상세주소'] == add]['위도'].values[0]
    marker = folium.Marker([lon1,lat1],popup='1지진옥외').add_to(map)
    marker = folium.Marker([lon2,lat2],popup='2지진해일').add_to(map)
map
#google map / naver map(면적 계산) 을 이용해 보았을 때 시설면적(지진옥외)가 더 정확한 것 같다. ex) 포항에 있는 환호공원)

In [32]:
#지진해일 데이터로 지우기.
df_total = df_total.drop(index = duplicated_list, axis = 0)
df_total.reset_index(drop='index', inplace = True)

## 데이터 저장

In [34]:
df_total.to_csv('지진대피소 종합본.csv',encoding='euckr',index=False)

앞으로 해야 할 것.